In [ ]:
import tokenize
import re
from tqdm import tqdm

input_path = r"C:\Users\epw268\Documents\addinda_text_summ.txt"
output_path = r"C:\Users\epw268\Documents\addinda_text_summNEW.txt"

triple_re = re.compile(r'^[urbfURBF]*(?:"""|\'\'\')')

with open(input_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
mask = [bytearray(b"\x01" * (len(line))) for line in lines]

with open(input_path, 'rb') as f:
    tokens = tokenize.tokenize(f.readline)
    for tok in tqdm(tokens, desc="Scanning tokens"):
        toknum, tokval, start, end, _ = tok
        if toknum == tokenize.COMMENT:
            line_idx, col = start
            for i in range(col, len(lines[line_idx-1])):
                mask[line_idx-1][i] = 0
        elif toknum == tokenize.STRING and triple_re.match(tokval):
            (s_line, s_col), (e_line, e_col) = start, end
            for ln in range(s_line-1, e_line):
                start_col = s_col if ln == s_line-1 else 0
                end_col = e_col if ln == e_line-1 else len(lines[ln])
                for i in range(start_col, end_col):
                    mask[ln][i] = 0

cleaned_lines = []
for ln, line in enumerate(lines):
    cleaned_line = ''.join(ch for i, ch in enumerate(line) if mask[ln][i])
    cleaned_lines.append(cleaned_line)

with open(output_path, 'w', encoding='utf-8') as f:
    f.writelines(cleaned_lines)